# FUNCIONAMIENTO

## Experiencias

### Reglas de funcionamiento

* Todas las experiencias son consideradas actividades en el flujo (stream)
* La aplicación permite sólo la publicación de experiencias. No se permite publicar fotos aisladas, textos ni videos que no formen parte de la experiencia
* Las experiencias son seguidas por los ususarios que explícitamente hagan click al botón 'Seguir'
* Para dejar de seguir una experiencia se hace clic al botón 'Dejar de seguir'



# DJANGO
## Instalación
(Pendiente)

## Virtualenv
(Pendiente)

## Packages

### stream-django
### django-macros
### multiselectfield
Usado para almacenar en la DB caracteres de los checkboxes, no cadenas que representan listas. 

(Pendiente)
<hr>

# BASE DE DATOS

## Reestablecer la DB (local)

(https://stackoverflow.com/questions/2056876/postgres-clear-entire-database-before-re-creating-re-populating-from-bash-scr)

Para reestablecer la DB localmente recordar detener el servidor de Django, de otra forma indicará que otra aplicación está usando la DB.

### Tirar la DB

    $ dropdb hf

O:

    $ sudo -u postgres dropdb DATABASE_NAME

También es posible entrar al shell de Postgres y correr comandos SQL

### Crear la DB

Las tablas de la DB se agrupan bajo las siguientes tres categorías:

- #### Modelos Django
    Commando para eliminar los archivos de migración actuales
    reset_mig 
- #### Ciudades
    (https://github.com/yourlabs/django-cities-light)
    Generar las tablas de países (sólo para propósitos de desarrollo)
        $ ./manage.py cities_light
    El proceso indica errores por algunos registros que presentan problemas de integridad. Ignorarlos.
- #### PPP
    Importar los CSV del PPP del Banco Mundial (Archivo ppp-gdp.csv). Seleccionar 'Create new table' antes de tratar e importar.

    (https://data.worldbank.org/indicator/pa.nus.ppp)
    Se importa el CSV usando un cliente para Postgres. 
    
    Se recomienda TablePlus (https://tableplus.com/)

#### Para crear la DB:

    $ createdb hf
    
También es posible crear la DB desde el shell de Postgres:

    $ psql
    postgres=# create database mydb;

#### Crear un superuser para la DB:

'user_name' es el nombre de usuario de un perfil con privilegios de administrador
    $ sudo -u user_name psql
    postgres=# create user myuser with encrypted password 'mypass';
    postgres=# grant all privileges on database mydb to myuser;

(https://link.medium.com/YdSIf0yTaZ)

#### Para correr comandos como super usuario en Postgres:

    $ su - postgres



## Reestablecer la DB (heroku)

- Paso 1: heroku restart
- Paso 2: heroku pg:reset DATABASE (no hay necesidad de cambiar DATABASE)
- Paso 3: heroku run rake db:migrate
- Paso 4: heroku run rake db:seed (if you have seed)


Para subir el CVS que contiene el PPP usar TablePlus y su opción importar. Para crear una conexión a DB de Heroku consultar las credenciales en la cuenta de Heroku. 

https://gist.github.com/zulhfreelancer/ea140d8ef9292fa9165e


<hr>

# SERVICIOS AWS
Heroku **no** es un servicio para el almacenamiento de datos para aplicaciones y como consecuencia de esto elimina contenido cargado por el usuario cada 30 minutos. En otras palabras usa un mecanismo efímero de **almacenamiento de archivos que sube el usuario**. Esto no significa que lo que se almacena en la DB sea efímero, pero ciertos datos del usuario (foto de perfil, fotos de la experiencia, etcétera) es necesario usar para eso un servicio especializado como el de Amazon (AWS).
<hr>

# HEROKU
Los buildpacks son scripts de instalación para Heroku. Para HF se usa el buildpack de CheesecakeLabs (https://github.com/CheesecakeLabs/heroku-geo-buildpack.git). Para instalar el buildpack:

    $ heroku buildpacks:set https://github.com/CheesecakeLabs/heroku-geo-buildpack.git
    (Para instalar Python)
    $ heroku buildpacks:add heroku/python

### Creación de una cuenta

### Instalación y uso básico de la línea de comandos (CLI)

### Loggin
    $ heroku logs
<hr>

# SOLUCIÓN DE PROBLEMAS


### <font color="red">La vista se llama mútiples veces o la vista no salva cambios del objeto</font>
Considerar que Django utiliza 'forked process' que son dos procesos, uno clon del otro, para el manejo de multitareas. Esto significa que los prints() dentro de los \__init__() de los objetos aparecerán duplicados, es el funcionamiento normal. Si se cree que hay una sobrecarga de proceso por doble instanciación de objetos intentar resolverlo de estas dos maneras:

* Causa 1: ocasionado por problemas en la referencia a archivos estáticos desde la plantilla HTML, una referencia None o duplicar llamadas a LINK o SCRIPT duplicadas.

* Causa 2: por llamadas a \__init()__ duplicadas en herencia múltiple. Revisar que la clase que hereda no haga llamadas dobles a \__init()__ usando super()

### <font color="red">Error: django.core.exceptions.AppRegistryNotReady: Apps aren't loaded yet.</font>

La aplicación no está lista. El manual recomienda importar las librerías dentro de app.py dentro del método ready() para evitar este problema de sincronizacíon, ejemplo:
    
    #El único import fuera de método ready()
    from django.apps import AppConfig

    class UsersConfig(AppConfig):
        name = 'usuarios'

        def ready(self):
            # Sugiere la documentación importar así para evitar conflictos
            import usuarios.signals
            from actstream import registry
            from django.contrib.auth.models import User
            registry.register(User,self.get_model('Experiencias'),self.get_model('User'))


### <font color="red">Tras registro el usuario recibe el correo de confirmación pero no logra firmarse</font>

El usuario no está activado por un problema con la liga de confirmación. Usuarios no activados no se pueden firmar. Es muy probable que la liga de confirmacón esté apuntando a sitio default example.com (ver solución abajo).

### <font color="red">La liga de activación de una nueva cuenta apunta al sitio example.com y no es posible activarla</font>

El administrador del Django permite hacer altas, bajas y cambios de dominios válidos que se guardan en una tabla de la DB. Después de borrar la DB hay que reasignar el 'site', creando uno nuevo y luego consultando en la DB, en la tabla de sitios el ID para asignarlo en settings.py la variable SITE_ID a ese ID.

    SITE_ID = <ID>


### <font color="red">La conexión a la DB con TablePlus a Heroku es rechazada</font>

Ocurre cuando se usan lo settings anteriores, se llenan los campos y podrían quedar caracteres inválidos. Lo más sencillo es tirar el perfil y crear uno nuevo. Consultar las credenciales desde www.heroku.com.

### <font color="red">Las imágenes en iOS aparecen deformadas (sólo iOS)</font>

Siempre utilizar la estructura .container>.row>.col (Bootstrap) para evitar problemas de acomodo en iOS

    				<div class="container">
						<div class="row">
							<div class="col"></div>
						</div>
					</div>
                    
### <font color="red">Servidor de Django indica error 500</font>

El servidor de Django está disponible sólo para desarrollo y espera que la variable DEBUG en settings.py tenga el valor True.

### <font color="red">El archivo requirements.txt contiene referencias a paquetes equivocados</font>

Asegurarse de estar en el directorio raiz de la aplicación (opcional) y que el ambiente virtual correspondiente esté activo (venv).

### <font color="red">'Reverse accessor for clashes with reverse accessor for'</font>

Hacer justo lo que indica el HINT: agregar a la declaración del campo en models.py un 'related_name' o bien cambiar su valor a uno distinto para evitar el choque de nombres:
	
    HINT: Add or change a related_name argument to the definition for 'Experiencia.tags' or 'Experiencia.tags'.
    
Nota: los campos que presentan este error son sólo relaciones Uno a Muchos y Muchos a Muchos
    
### <font color="red">¿Cómo se borra el caché de Heroku?</font>

    $ heroku plugins:install heroku-repo
    $ heroku repo:purge_cache -a appname
    $ git commit --allow-empty -m "Purge cache"
    $ git push heroku master

https://help.heroku.com/18PI5RSY/how-do-i-clear-the-build-cache

### <font color="red">Los campos requeridos indicados desde el modelo (sin blank=true) no generan ValidationError</font>

Los campos del modelo indicados como requeridos (sin blank=true) son validados desde el modelo aún cuando se defina un método específico para validarlos en forms.py. Si esto es así, cuando se llama desde views.py form.is_valid() éste método no llama a su vez al método clean() de forms.py porque fue validado desde el modelo. Las opciones son validar desde el modelo o indicar el campo en el modelo con la bandera blank=true.

### <font color="red">El campo ImageField no genera errores de validación
    
Tras intentar la misma estrategia que se usó con otros campos indicando blank=True y validando en la forma, si el campo no pertenece a la misma forma y desde el template no se indica explicitamente que se muestren errores (form.errors) Django empleará su validación básica (un globo con indicaciones escuetas aparece en los campos con error).

### <font color="red">Input se desactiva
    
La directiva 'as_hidden' inhabilita los inputs del html, así:

{{ form.field.as_hidden }}

Emplear otro recurso como usar d-none de Bootstrap

### <font color="red">LookupError at /user/admin/ App 'appname' doesn't have a 'modelname' model.
    
    
Es sumamente importante que cuando se hace una referencia cruzada entre modelos, la importación del modelo que se referirá en el modelo actual <b>SE INDIQUE HASTA ARRIBA</b>. Ejemplo:

    from django.db import models
    from geography.models import ZipCode

    class Restaurant(models.Model):
        # ...
        zip_code = models.ForeignKey(
            ZipCode,
            on_delete=models.SET_NULL,
            blank=True,
            null=True,
        )